In [1]:
import pdfplumber

file_path = 'guardian.pdf'      

In [2]:
with pdfplumber.open(file_path) as pdf:
  
    for page in pdf.pages:
        tables = page.extract_tables()
        t = 1
        for table in tables:
            print("\n")
#             print("Table", t)
            t += 1
            r = 1
#             print(len(table))
#             print(table[2])    
            for row in table[:-1]:
#                 print("Row", r)
#                 print(row)
                
                r +=1
                if 'Claim Number' in row[0]:
                    text = row[0]
                    
                if 'Line' in row[0]:
                    header = row
#                     print("Header", header)
                

In [3]:
text

'Claim Number: 32883F15100 Patient Account No.:121148666 Plan Number:00518516\nPatient Name: TREYTON ROPER Employee Name: BLAKE ROPER Relationship: SON\nPlanholder: ONBOARDIQ, INC.'

In [4]:
# text = 'Claim Number: 18472E05500 Patient Account No.:15320 Plan Number:00580287 Patient Name: HARPER L NEWTON Employee Name: MICHELLE N BERMAN Relationship: DAUGHTER Planholder: GDH CONSULTING, INC.'

In [5]:
claimnumber = text.split('Claim Number: ')[1].split(' ')[0]
patientaccountno = text.split('Patient Account No.:')[1].split(' ')[0]
plannumber = text.split('Plan Number:')[1].split(' ')[0]
patientname = text.split('Patient Name: ')[1].split('Employee Name')[0]
employeename = text.split('Employee Name: ')[1].split('Relationship')[0]
relationship = text.split('Relationship: ')[1].split('Planholder:')[0]
planholder = text.split('Planholder: ')[1]
print(claimnumber)
print(patientaccountno)
print(plannumber)
print(patientname)
print(employeename)
print(relationship)
print(planholder)

32883F15100
121148666
00518516
Patient
TREYTON ROPER 
BLAKE ROPER 
SON

ONBOARDIQ, INC.


In [6]:
header = ['Line\nNo.', 'Submitted\nADA Codes/Description', 'Alt\nCode', 'Tooth\nNo.', 'Date of\nService', 'Submitted\nCharge', 'Considered\nCharge', 'Covered\nCharge', 'Deductible\nAmount', 'Coverage\nPercent', 'Benefit\nAmount']

In [7]:
head = []
for h in header:
    if 'Line' in h:
        head.append("LineNo")
    if 'Submitted\nADA' in h:
        head.append("SubmittedADACodesDescription")
    if 'Alt' in h:
        head.append("AltCode")
    if 'Tooth' in h:
        head.append("ToothNo")
    if 'Date of' in h:
        head.append("DateOfService")
    if 'Submitted\nCharge' in h:
        head.append("SubmittedCharge")
    if 'Considered' in h:
        head.append("ConsideredCharge")
    if 'Covered\nCharge' in h:
        head.append("CoveredCharge")
    if 'Deductible' in h:
        head.append("DeductibleAmount")
    if 'Coverage\nPercent' in h:
        head.append("CoveragePercent")
    if 'Benefit\nAmount' in h:
        head.append("BenefitAmount")
head      
        
        

['LineNo',
 'SubmittedADACodesDescription',
 'AltCode',
 'ToothNo',
 'DateOfService',
 'SubmittedCharge',
 'ConsideredCharge',
 'CoveredCharge',
 'DeductibleAmount',
 'CoveragePercent',
 'BenefitAmount']

In [8]:
import tabula
import pandas as pd
tabula_dfs = tabula.read_pdf(file_path,guess=False,pages=1,stream=True,encoding="utf-8",area=(141, 43, 176, 217),multiple_tables=True)

Got stderr: Jul 13, 2023 4:20:12 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jul 13, 2023 4:20:12 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jul 13, 2023 4:20:12 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jul 13, 2023 4:20:12 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jul 13, 2023 4:20:12 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jul 13, 2023 4:20:12 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jul 13, 2023 4:20:12 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding



In [9]:
tabula_dfs[0]

,UT BCDO SPECIALTY DENTAL SERVI
0,3401 N CENTER ST STE 250
1,"LEHI, UT 84043-7501"


In [10]:
def process_tabula_address(tabula_df: pd.DataFrame) -> str:
    col_name: str = tabula_df.columns[0]
    item: str = f"{col_name} {' '.join(tabula_df[col_name].to_list())}"
    return item

In [11]:
payee_address = process_tabula_address(tabula_dfs[0])

In [12]:
from Utilities.pdf_utils  import *
def extract_address_details(address_text):
    try:
        address_list = usaddress.tag(address_text)[0]
    except usaddress.RepeatedLabelError as e:
            parsed_address = e.parsed_string
            print('parsed address is for payor is----------', parsed_address )
            address_list = get_address_list_when_exception(parsed_address)


    return {
        'Item': address_text,
        'ItemName': address_list.get('Recipient', ''),
        'AddressElements': get_address(address_text, address_list),
        'PlaceName': address_list.get('PlaceName', ''),
        'StateName': address_list.get('StateName', ''),
        'ZipCode': address_list.get('ZipCode', '')
    }

In [13]:
payee_add_details = extract_address_details(payee_address)

In [14]:
payee_add_details

{'Item': 'UT BCDO SPECIALTY DENTAL SERVI 3401 N CENTER ST STE 250 LEHI, UT  84043-7501',
 'ItemName': 'UT BCDO SPECIALTY DENTAL SERVI',
 'AddressElements': '3401 N CENTER ST STE 250',
 'PlaceName': 'LEHI',
 'StateName': 'UT',
 'ZipCode': '84043-7501'}

In [15]:
tabula_dfs = tabula.read_pdf(file_path,guess=False,pages=1,stream=True,encoding="utf-8",area=(25, 90, 53, 213),multiple_tables=True)

Got stderr: Jul 13, 2023 4:20:13 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jul 13, 2023 4:20:13 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jul 13, 2023 4:20:13 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jul 13, 2023 4:20:13 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jul 13, 2023 4:20:13 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jul 13, 2023 4:20:13 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jul 13, 2023 4:20:13 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding



In [16]:
payer_address = process_tabula_address(tabula_dfs[0])

In [17]:
payer_address

'PO BOX 981572 EL PASO, TX  79998-1572'

In [18]:
payer_add_details = extract_address_details(payer_address)

In [19]:
payer_add_details

{'Item': 'PO BOX 981572 EL PASO, TX  79998-1572',
 'ItemName': '',
 'AddressElements': 'PO BOX 981572',
 'PlaceName': 'EL PASO',
 'StateName': 'TX',
 'ZipCode': '79998-1572'}

In [20]:
other_info = tabula.read_pdf(file_path,guess=False,pages=1,stream=True,encoding="utf-8",area=(97, 288, 152, 570),multiple_tables=True)

Got stderr: Jul 13, 2023 4:20:14 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jul 13, 2023 4:20:14 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jul 13, 2023 4:20:14 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jul 13, 2023 4:20:14 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jul 13, 2023 4:20:14 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jul 13, 2023 4:20:14 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jul 13, 2023 4:20:14 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding



In [21]:
df = other_info[0]

In [22]:
df = df.T.reset_index()
df.columns = df.iloc[0]
df = df[1:].reset_index(drop=True)

In [23]:
dit = df.to_dict(orient='records')

In [24]:
dit = dit[0]

In [25]:
dit

{'Provider:': 'DAVID HADLEY',
 'Date:': '05/31/2023',
 'Payee:': 'UT BCDO SPECIALTY DENTAL SER',
 'Check No.:': '188218572',
 'Payment Amount:': '$160.00'}

In [26]:
from PyPDF2 import PdfReader
texts = ' '
with open(file_path, 'rb') as file:
    # Create a PdfReader object
    pdf = PdfReader(file)

    # Extract text from each page
    for page in pdf.pages:
        text = page.extract_text()
        texts = texts + text + ' '
 

C:\Users\pytho\AppData\Roaming\Python\Python310\site-packages\PyPDF2\_cmap.py:142: PdfReadWarning: Advanced encoding /BuiltIn not implemented yet
  warnings.warn(


In [27]:
texts

" If you have any\nquestions contact:(800) 541-7846\nWWW.GUARDIANANYTIME.COM\nProvider: DAVID HADLEY\nDate: 05/31/2023\nPROVIDER EXPLANATION OF BENEFITS - THIS IS NOT A BILL\nImportant!  Please examine this statement for accuracy.  Save this statement for tax purposes.GROUP PLAN ADMINISTRATORS\nPayee:\nCheck No.:UT BCDO SPECIALTY DENTAL SER\n188218572\nPayment Amount: $160.00\nExpedite cash flow with e-payments. Sign up today!\nGuardian has contracted with Change Healthcare, a leading provider of \nrevenue and payment cycle solutions, to deliver Electronic Funds Transfer \n(EFT) services! Sign-up today by going to www.changehealthcare.com/support\n/customer-resources/enrollment-services  for more information. Enrollment for \nthis service is offered to you at no additional cost and is available online or by \ncalling 1.866.777.0713 and selecting Option 1.\nSubmitted\nADA Codes/DescriptionTooth\nNo.Date of\nServiceSubmitted\nChargeCoverage\nPercentAlt\nCodeConsidered\nChargeCovered\nCha

In [28]:
# claimnumber = '18472E05500'
claimnumber = '36347F13200'
remarks = ''
remarks = texts.split(f"Remarks for claim # {claimnumber}:")[1]

if '10 Hudson' in  remarks:
    remarks = remarks.split('10 Hudson')[0]
else:
    remarks = remarks.split('Comments:')[0]

IndexError: list index out of range

In [ ]:
remarks.replace('\n', ' ').strip()

In [ ]:
benefit = texts.split(f"Remarks for claim # {claimnumber}:")[0]
if 'Remarks for claim' in benefit:
    benefit = benefit.split('Remarks for claim')[1]
benefit

In [ ]:
paidbyotherinsu = '$'+ benefit.split('PAID BY OTHER INSURANCE')[1].split('ADJUSTMENTS')[0].replace('\n','').split('$')[1]
paidbyotherinsu

In [ ]:
adj = '$'+ benefit.split('ADJUSTMENTS')[1].split('TOTAL BENEFIT PAID')[0].replace('\n','').split('$')[1]
adj

In [ ]:
totalbenefitpaid = '$'+ benefit.split('TOTAL BENEFIT PAID')[1].split('PATIENT')[0].replace('\n','').split('$')[1]
totalbenefitpaid

In [ ]:
patientrep = '$'+ benefit.split('PATIENT')[1].split('TOTALS\nTOTAL BENEFIT')[0].replace('\n','').split('$')[1].strip()
patientrep

In [ ]:
toalbenfitpayable = benefit.split('PAID BY OTHER INSURANCE')[0].split('\n')[-2].strip()
toalbenfitpayable

In [ ]:
higherallowable = '$'+ benefit.split('BENEFIT SUMMARY')[1].split('HIGHER ALLOWABLE')[0].replace('\n','').split('$')[1].strip()
higherallowable

In [ ]:
# Getting master details for other design of pdf


In [36]:
import pdfplumber
from PyPDF2 import PdfReader
file_path = 'guardian01.pdf'  

url = ''

In [37]:
def getAllTexts(file_path):
    texts = ' '
    with open(file_path, 'rb') as file:
        # Create a PdfReader object
        pdf = PdfReader(file)

        for page in pdf.pages:
            text = page.extract_text()
            texts = texts + text + ' '
    return texts

In [38]:
texts = getAllTexts(file_path)

In [39]:
texts

' Claim Number: 37856F16600\nSubmitted\nADA Codes/DescriptionTooth\nNo.Date of\nServiceSubmitted\nChargeCoverage\nPercentIf you have any\nquestions contact:(800) 541-7846\nWWW.GUARDIANANYTIME.COM\nEmployee Name: JARED MELLOR\nPatient Name: KOLSEN MELLOR\nRelationship: SON\nPatient Account No.: 121748878\nPlan Number: 00025210\nPlanholder: ALARM.COM, INC.\nProvider: JEFFREY T WELCH, DMD\nDate: 06/16/23\nAlt\nCodeConsidered\nChargeCovered\nChargeDeductible\nAmountBenefit\nAmountEXPLANATION OF BENEFITS - THIS IS NOT A BILL\nImportant! Save this statement for tax purposes.\nLine \nNo.\nD2332/Composite 3 Sr 102.00 102.00 06/14/23 91.80 172.00 90% 09 1\nD2332/Composite 3 Sr 102.00 102.00 06/14/23 91.80 172.00 90% 10 2\n344.00 204.00 204.00 183.60\n$183.60\nPAID BY OTHER INSURANCE..... $0.00\nADJUSTMENTS............................... $0.00\nTOTAL BENEFIT PAID.................. $183.60\nPATIENT\'S RESPONSIBILITY\n....................................$20.40TOTALS\nTOTAL BENEFIT PAYABLE.........

In [40]:
def get_patient_dict(text):
    patient_dict = {}
    claimnumber = text.split('Claim Number: ')[1].split(' ')[0]
    patientaccountno = text.split('Patient Account No.:')[1].split(' ')[0]
    plannumber = text.split('Plan Number:')[1].split(' ')[0].split('\n')[0]
    patientname = text.split('Patient Name: ')[1].split('Employee Name')[0].strip()
    employeename = text.split('Employee Name: ')[1].split('Relationship')[0].strip()
    relationship = text.split('Relationship: ')[1].split('Planholder:')[0].replace('\n', '')
    planholder = text.split('Planholder: ')[1]

    patient_dict['SubscriberID'] = ''
    patient_dict['PatientName'] = patientname
    patient_dict['Relationship'] = relationship
    patient_dict['ClaimId'] = claimnumber
    patient_dict['PatientAccount'] = patientaccountno
    patient_dict['PlanNumber'] = plannumber
    patient_dict['SubscriberName'] = employeename
    patient_dict['PlanType'] = planholder
    patient_dict['TransactionFee'] = '$0.00'
    patient_dict['url'] = url
    patient_dict['PayerClaimID'] = ''
    patient_dict['TotalAmount'] = ''
    patient_dict['ClaimStatus'] = 'Processed'
    patient_dict['RenderingProvider'] = ''
    patient_dict['PayerClaimID'] = claimnumber
    
    return patient_dict
    

In [41]:
def get_master_details(file_path, texts, url):
    with pdfplumber.open(file_path) as pdf:
        patients = []
        for page in pdf.pages:
            tables = page.extract_tables()
            t = 1
            for table in tables:

                t += 1
                r = 1
                for row in table[:-1]:
                    r += 1
                    if 'Claim Number' in row[0]:
                        patient_dict = {}
                        text = row[0]
                        patient_dict = get_patient_dict(text)
                        
#                         payee_address_details = get_basic_details(file_path)
#                         notes = getRemarks(texts, claimnumber)
#                         patient_dict.update(payee_address_details)
                        
#                         patient_dict['Notes'] = notes
#                         benefit, adjustments = getBenefit(texts, claimnumber)
#                         patient_dict.update(benefit)

                        patient_dict['PPGridViewId'] = '6'
                        patient_dict['PPTransPayorListID'] = "27e7c674-051c-40ec-b9ef-6c84f3a3dd1d"
                        patient_dict['PayeeTaxID'] = ''
                        patient_dict['PayerContact'] = '(800) 541-7846'
                        patient_dict['PayerID'] = ''
                        patient_dict['PaymentMethodCode'] = ''
#                         patient_dict['RecordID'] = str(uuid.uuid4())
                        patient_dict['RenderingProviderID'] = ''

                        patients.append((patient_dict))
                    else:
                        print("NO CLAIM NUMBER")

    pl = len(patients)
    indexlist = []
    for i in range(pl):
        try:
            for j in range(i + 1, pl + 1):
                try:
                    if patients[i]["ClaimId"] == patients[j]["ClaimId"]:
                        indexlist.append(j)
                except:
                    pass
        except:
            pass

    indexlist.sort(reverse=True)

    for index in indexlist:
        del patients[index]

    return patients

In [45]:
import tabula
import pandas as pd
def get__other_master_details(file_path, texts, url):
    master_info = tabula.read_pdf(file_path,guess=False,pages=1,stream=True,encoding="utf-8",area=(110, 289, 222, 524),multiple_tables=True)
    print(master_info)

In [46]:
if 'Claim Number' in texts.split('\n')[0]:
    eobclaimmaster = get__other_master_details(file_path, texts, url)
else:
    eobclaimmaster = get_master_details(file_path, texts, url)

# eobclaimmaster

Got stderr: Jul 13, 2023 4:22:00 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jul 13, 2023 4:22:00 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jul 13, 2023 4:22:00 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jul 13, 2023 4:22:00 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jul 13, 2023 4:22:00 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jul 13, 2023 4:22:00 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding



[        Claim Number: 37856F16600
0     Employee Name: JARED MELLOR
1     Patient Name: KOLSEN MELLOR
2               Relationship: SON
3   Patient Account No.:121748878
4           Plan Number: 00025210
5     Planholder: ALARM.COM, INC.
6  Provider: JEFFREY T WELCH, DMD
7                  Date: 06/16/23]
